Read and preprocess the lyrics of Thornapple

In [1]:
import pandas as pd
# import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')


df = pd.read_csv('C://Users//Jinsoon//Desktop//NLP with Python//Lyrics_Thornapple_mod1.csv', 
                 engine='python', error_bad_lines=False);
# engine='python' is needed to solve the 'utf-8' encoding error. Thanks to http://kkckc.tistory.com/187 

df_text = df[['Lyrics']]
df_text['index'] = df_text.index
docs = df_text

In [2]:
len(docs)
# total 25 songs excluding instrumentals. Need to drop 3.

29

In [3]:
docs = docs.dropna()

In [4]:
# Check first 5 rows
docs[:5]

,Lyrics,index
0,이젠 까마득해요 온전한 당신을 먹은 기억 여긴 날씨가 좋아요 이젠 별로 열도 안 나...,0
1,지도에 없는 곳으로 가려고 집을 나선 날 바람이 몹시도 불었네 그대론 어디로도 갈 ...,1
2,그대는 내겐 너무도 어려운 달 내겐 너무도 어려운 달이었어요 우리는 서로 너무도 ...,2
3,태어나지도 않은 아이가 멀리서 울부짖는 시간은 언제나 그 끝을 알 수 없는 어스름녘...,3
4,터지는 기침을 참고 한 번도 원한 적 없는 가파른 숨을 시작했던 날에 한 뼘의 흙마...,4


In [5]:
from konlpy.tag import Twitter, Kkma, Komoran
twitter = Twitter()
kkma = Kkma()
komo = Komoran()

In [25]:
def tokenizer_kkma_morphs(doc):
    return kkma.morphs(doc)

def tokenizer_twitter_morphs(doc):
    return twitter.morphs(doc)

def pos_tagger(doc):
    return twitter.pos(doc)

def tokenizer_komoran_morphs(doc):
    return komo.morphs(doc)

In [26]:
docs['Tokened_Lyrics'] = docs['Lyrics'].apply(tokenizer_twitter_morphs)

C:\Users\Jinsoon\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
docs['Tokened_Lyrics'][0:3]

0    [이, 젠, 까마, 득, 해, 요, 온전, 한, 당신, 을, 먹은, 기억, 여긴, ...
1    [지도, 에, 없는, 곳, 으로, 가려, 고, 집, 을, 나선, 날, 바람, 이, ...
2    [그대, 는, 내겐, 너무, 도, 어려운, 달, 내겐, 너무, 도, 어려운, 달, ...
Name: Tokened_Lyrics, dtype: object

In [9]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim') # suppress the chunkize warning 
from gensim.models import LdaModel, HdpModel
from gensim.corpora.dictionary import Dictionary
import pyLDAvis.gensim
import os

In [10]:
def get_corpus_dictionary():

    corpus = docs['Tokened_Lyrics']
    dictionary = Dictionary(corpus)

    # Transforming corpus with dictionary.
    corpus = [dictionary.doc2bow(doc) for doc in corpus]

    # Building reverse index.
    for (token, uid) in dictionary.token2id.items():
        dictionary.id2token[uid] = token

    return corpus, dictionary

LDA model training 

In [11]:
corpus, dictionary = get_corpus_dictionary()

In [12]:
lda = LdaModel(corpus=corpus, num_topics=4)

In [13]:
data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [14]:
pyLDAvis.save_html(data, 'index_lda.html')
os.remove('index_lda.html')

In [15]:
pyLDAvis.display(data)